**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [1]:
import gzip
import requests
from bs4 import BeautifulSoup

In [2]:
# url='http://lib.ru/NEWPROZA/LOBAS/taxisty.txt'
# text = BeautifulSoup(requests.get(url).text).get_text()
# with gzip.open('taxisty.txt.gz','wt') as f: f.write(text)

# # with gzip.open('taxisty.txt.gz','rt') as f: text = f.read()

# text = text[1030:-7261].strip() # выкидываем заголовок и хвост страницы 
# print(f'символов:{len(text)}\n---------------\n'%())
# print(text[:343])

In [3]:
url='http://az.lib.ru/d/dostoewskij_f_m/text_0080.shtml'
text = BeautifulSoup(requests.get(url).text).get_text()
with gzip.open('dostoewskij.txt.gz','wt') as f: f.write(text)

# with gzip.open('dostoewskij.txt.gz','rt') as f: text = f.read()

text = text[2876:-664184].strip() # выкидываем заголовок и хвост страницы 
print(f'символов:{len(text)}\n---------------\n'%())
print(text[:355])

символов:1279540
---------------

Приступая к описанию недавних и столь странных событий, происшедших в нашем, доселе ничем не отличавшемся городе, я принужден, по неумению моему, начать несколько издалека, а именно некоторыми биографическими подробностями о талантливом и многочтимом Степане Трофимовиче Верховенском. Пусть эти подробности послужат лишь введением к предлагаемой хронике, 


In [4]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [5]:
from tqdm.auto import tqdm
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

sentences = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in tqdm(nltk_sentence_split(text)) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(sentences)))
display( sample(sentences,1) )

  0%|          | 0/14574 [00:00<?, ?it/s]

предложений: 14574



[['Та',
  'глядела',
  'на',
  'нее',
  'ужасно',
  'оробевшим',
  ',',
  'застыдившимся',
  ',',
  'но',
  'почти',
  'благоговейным',
  'взглядом',
  'и',
  'вдруг',
  'усмехнулась',
  'с',
  'тем',
  'же',
  'странным',
  'хихиканьем',
  '.']]

In [6]:
sentences = sentences[:1024*7] # ограничиваем датасет для ускорения процеса 

In [7]:
%%time

from nltk.lm.preprocessing import padded_everygram_pipeline 

ngram_len = 2

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(ngram_len, sentences)

CPU times: user 10 µs, sys: 2 µs, total: 12 µs
Wall time: 14.1 µs


In [8]:
# собираем модель

# from nltk.lm import MLE as LangModel 
from nltk.lm import Laplace as LangModel

model = LangModel(ngram_len) 
model.fit(train, vocab)

display(len(model.vocab))

20835

In [9]:
# генерируем продолжения
for sentence in sample(sentences,30): # выбираем рандомно 10 предложений
    if len(sentence)<10: continue
    # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    # генерируем возможные продолжения
    result = model.generate(3, text_seed=sentence_) 
    print(  ' '.join(sentence_)  + ' ... ' + str( result ) + '\n' )

Стоя на коленях , придавленный к земле , с вывернутыми на спину локтями , хитрый бродяга спокойно ожидал развязки , совершенно , ... ['когда', 'наконец', 'спросить']

Он на месяц приедет , последнее имение продавать здесь ... ['ей', 'обещал', 'зайти']

Но истина одна , а стало быть , только единый из народов и может иметь бога истинного , хотя бы остальные народы и ... ['прискакивая', ',', 'тогда']

Поднялся смех , ропот ; толпа стеснилась , стали ругаться , дошло бы и до побоев , если ... ['противник', 'заранее', 'все']

Заметно было , что mademoiselle Лебядкина белится и румянится и губы ... ['были', 'так', 'процветавшего']

Я , знаете , устраиваю целый день увеселений , по подписке , в пользу бедных ... ['.', '</s>', 'Боже']

А что если подымется крик и вместо величественной картины дойдет ... ['до', 'генерала', '.']

- Если не к вам , то я пойду в сестры милосердия , в сиделки , ходить за больны ми , ... ['он', 'ни', 'Горацио']

Еще раз повторяю : я и тогда считал его и теперь счит

In [12]:
# from nltk.util import bigrams
from nltk.util import ngrams

# оцениваем насколько хорошо модель предсказывает слова из датасета
text_ngrams = [ ng for s in sentences for ng in ngrams(s,ngram_len) ]

print( 'perplexity:', model.perplexity( text_ngrams ) )

perplexity: 4018.948842449797


In [13]:
display( text_ngrams[:3] )

[('Приступая', 'к'), ('к', 'описанию'), ('описанию', 'недавних')]